In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
spark = SparkSession.builder \
    .appName('Homogenizar datos') \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
## Cambiar la ruta de los datos

In [3]:
df = spark.read.option("delimiter", ",").option("header", True).csv("../../../data/raw/data_aemet/estacion_3195_data.csv")

In [4]:
df.columns

['fecha',
 'indicativo',
 'nombre',
 'provincia',
 'altitud',
 'tmed',
 'prec',
 'tmin',
 'horatmin',
 'tmax',
 'horatmax',
 'dir',
 'velmedia',
 'racha',
 'horaracha',
 'presMax',
 'horaPresMax',
 'presMin',
 'horaPresMin',
 'hrMedia',
 'hrMax',
 'horaHrMax',
 'hrMin',
 'horaHrMin']

In [20]:
df = df.withColumnRenamed("tmed", "temperatura_media") \
       .withColumnRenamed("hrMedia", "humedad_media") \
       .withColumnRenamed("tmax", "temperatura_maxima") \
       .withColumnRenamed("tmin", "temperatura_minima")

In [21]:
df = df.withColumn("temperatura_media", F.regexp_replace(F.col("temperatura_media"), ",", "."))
df = df.withColumn("humedad_media", F.regexp_replace(F.col("humedad_media"), ",", "."))
df = df.withColumn("temperatura_maxima", F.regexp_replace(F.col("temperatura_maxima"), ",", "."))
df = df.withColumn("temperatura_minima", F.regexp_replace(F.col("temperatura_minima"), ",", "."))

In [22]:
df = df.withColumn("fecha", F.to_date(df["fecha"], "yyyy-MM-dd"))

In [23]:
df = df.withColumn("temperatura_media", F.col("temperatura_media").cast("float"))
df = df.withColumn("humedad_media", F.col("humedad_media").cast("float"))
df = df.withColumn("temperatura_maxima", F.col("temperatura_maxima").cast("float"))
df = df.withColumn("temperatura_minima", F.col("temperatura_minima").cast("float"))

In [24]:
df = df.withColumn("anio", F.year(df["fecha"]))
df = df.withColumn("mes", F.month(df["fecha"]))

In [25]:
df_temp_media = df.select(F.col("fecha"), F.col("anio"), F.col("mes"), F.col("temperatura_media"))
df_humedad = df.select(F.col("fecha"), F.col("anio"), F.col("mes"), F.col("humedad_media"))
df_temp_maxima = df.select(F.col("fecha"), F.col("anio"), F.col("mes"), F.col("temperatura_maxima"))
df_temp_minima = df.select(F.col("fecha"), F.col("anio"), F.col("mes"), F.col("temperatura_minima"))

In [11]:
df_temp_media.dtypes

[('fecha', 'date'),
 ('anio', 'int'),
 ('mes', 'int'),
 ('temperatura_media', 'float')]

In [26]:
base_dir = "../../../data/homogenized/aemet_data/humedad/"

In [27]:
for row in df_humedad.select("anio", "mes").distinct().collect():
    year_val = row["anio"]
    month_val = row["mes"]
    
    # Crear el directorio por año y mes
    dir_path = os.path.join(base_dir, f"Año_{year_val}/Mes_{month_val}")
    
    # Crear las carpetas si no existen
    os.makedirs(dir_path, exist_ok=True)
    
    # Filtrar el DataFrame para este año y mes
    df_filtered = df.filter((df_humedad["anio"] == year_val) & (df_humedad["mes"] == month_val))
    
    # Guardar los datos filtrados en la carpeta correspondiente (por ejemplo, en formato CSV)
    df_filtered.write.parquet(os.path.join(dir_path, f"estacion_3195_{year_val}_{month_val}.parquet"), mode="overwrite")

print("Datos guardados por año y mes.")

Datos guardados por año y mes.
